In [ ]:
import tensorflow as tf
from tensorflow.python.ops.rnn import _transpose_batch_time

from datetime import datetime
import numpy as np
import pandas as pd
import glob, os, sys, math, warnings
import matplotlib.pyplot as plt
import copy, time, glob, os, sys
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# customized ftns 
from helpers import *
from utilities import *
from model import *
from train import train_all_single_policies
# ---------------------------------------------------------
%matplotlib inline
%load_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
# ---------------------------------------------------------
# directories
main_dir = '../'
game_dir = main_dir+'data/'
Data = LoadData(main_dir, game_dir)
models_path = './models/' 

#### Load raw data

In [ ]:
# %%time
game_id = '0021500463'
game_data = Data.load_game(game_id)
events_df = pd.DataFrame(game_data['events'])
print('raw events shape:', events_df.shape)
events_df.head(3)

#### Get some suplementary data

In [ ]:
# play id to play roles/positions
id_role = id_position(events_df)
check_game_roles_duplicates(id_role)

# its possible that F has similar role as G-f or F-G, we create empty slots to ensure meta order
# ddentify defending and offending runs (this is included in process_moments)
court_index = Data.load_csv('./meta_data/court_index.csv')
court_index = dict(zip(court_index.game_id, court_index.court_position))

# home and visitor ids
homeid = events_df.loc[0].home['teamid']
awayid = events_df.loc[0].visitor['teamid']

#### Pre-process 
filter events, subsample frames, add velocity, reorder moments, re-arrange team order
shot clock, filter out event with short moments

In [ ]:
# filter out actions except 1: Make, 2: Miss, 4: Rebound, 6:Personal Foul, 7:Violation
use_event = [1, 2, 4, 6, 7]
discard_event = [3, 5, 8, 9, 10, 12, 13, 18]
events = filter_event_type(events_df, discard_event)
print('After filtering events has shape:', events.shape)
# break up sequences at 24secs shot clock point (or irregular case, e.g. out of bound maybe),
# and obtain the game data
subsample_factor = 0
single_game, single_game_balls = get_game_data_ra(events, court_index, game_id, 
                                                  event_threshold=10, subsample_factor=subsample_factor)
print('Final number of events:', len(single_game))

# get player velocity
fs_base = 1./25 # 1/25 sec/frame   or  25 frames/sec
fs = fs_base * subsample_factor if subsample_factor != 0 else fs_base
single_game = [get_velocity(i, fs, mode=1) for i in single_game]
n_events = len(single_game)

# get basketball velocity
single_game_balls = [np.concatenate([i[:-1, :], get_velocity(i, fs, mode=0)], axis=1) for i in single_game_balls]

In [ ]:
single_game[0].shape

In [ ]:
single_game_balls[0].shape

#### Role assignment and reorder moment

In [ ]:
# first prepare data
n_defend = 5
n_offend = 5
n_ind = 4

# length for each moment
event_lengths = np.array([len(i) for i in single_game])
# repeat the event_lengths 5 times in order to match the unstack later on with moments
event_lengths_repeat = np.concatenate([event_lengths for _ in range(n_defend)], axis=0)
# all the moments
all_moments = np.concatenate(single_game, axis=0)
all_moments_vel = np.concatenate(single_game, axis=0) # vel
# we only need the first 5 players x,y coordindates
# defend
all_defend_moments = all_moments[:, :n_ind*n_defend]
# offend
all_offend_moments = all_moments[:, n_ind*n_offend:]

# flattened
all_defend_moments_ = np.concatenate([all_defend_moments[:, i:i+n_ind] for i in range(0, n_ind*n_defend, n_ind)], axis=0)
all_offend_moments_ = np.concatenate([all_offend_moments[:, i:i+n_ind] for i in range(0, n_ind*n_offend, n_ind)], axis=0)

In [ ]:
# create hmm model
n_comp = 7
n_mix = None
RA = RoleAssignment(n_iter=50,verbose=True)

In [ ]:
# train
defend_state_sequence_, defend_means, defend_covs, _ = RA.train_hmm(all_defend_moments_, event_lengths_repeat, n_comp, n_mix)
offend_state_sequence_, offend_means, offend_covs, _ = RA.train_hmm(all_offend_moments_, event_lengths_repeat, n_comp, n_mix)
# get role orders
_, defend_roles = RA.assign_roles(all_defend_moments_, all_defend_moments, defend_means, event_lengths)
_, offend_roles = RA.assign_roles(all_offend_moments_, all_offend_moments, offend_means, event_lengths)

In [ ]:
defend_pos_vel = order_moment_ra([i[:, :n_ind*5] for i in single_game], defend_roles)
offend_pos_vel = order_moment_ra([i[:, n_ind*5:] for i in single_game], offend_roles)

In [ ]:
single_game_balls[0].shape

In [ ]:
defend_pos_vel[0].shape

In [ ]:
defend_pos_vel[0]

In [ ]:
# concatenate defend, offend roles pos and velocity and the basketball pos and vel
single_game = [np.concatenate([defend_pos_vel[i], offend_pos_vel[i], single_game_balls[i]], axis=1) for i in range(n_events)]
# single_game = [np.concatenate([defend_pos_vel[i], offend_pos_vel[i]], axis=1) for i in range(n_events)]

In [ ]:
single_game[0].shape

In [ ]:
all_roles = [np.concatenate([defend_roles[i], offend_roles[i]], axis=1) for i in range(len(single_game))]

In [ ]:
all_roles[0].shape

In [ ]:
len(all_roles)

In [ ]:
n_events

#### Show the plot, for the sake of comparison with processed moment later on

In [ ]:
# Plot = PlotGame(game_id, main_dir, game_dir)
# # for i in range(plotn): 
# Plot.load_moment2img(game_data, event_number=0, moment_number=0, return_img=True)

In [ ]:
# # manual plot check
# plot_check(single_game, plt_ind=0)

#### Build graph and starts training for all single policies

In [ ]:
policies = list(range(7))
single_policies = [ImportGraph('policy{}/'.format(str(i))) for i in policies]

In [ ]:
# model = ImportGraph('policy1/')
# result = model.run(train_x=val_xi, train_y=val_yi, seq_len=true_seq_len, h=k)
# result

In [ ]:
sequence_length = 30
overlap = 15
batch_size = 32
policy_targets = [np.array(get_sequences(single_game, policy, sequence_length, overlap)[1]) for policy in policies]
policy_targets = np.concatenate(policy_targets, axis=2)

In [ ]:
# policy_targets[6].shape

In [ ]:
# a = np.arange(12).reshape(2,3,2)
# a

In [ ]:
# b = np.arange(10,22).reshape(2,3,2)
# b

In [ ]:
# np.concatenate((a,b), axis=2)

In [ ]:
policy_targets[0][4].shape

At each epoch we will first do a forward pass to get the current step output, then we will use it for next roll-out position input and train

In [ ]:
sequence_length = 30
overlap = 15
batch_size = 32
policy_targets = [np.array(get_sequences(single_game, policy, sequence_length, overlap)[1]) for policy in policies]
policy_targets = np.concatenate(policy_targets, axis=2)
train = get_sequences(single_game, 0, sequence_length, overlap)[0] # all the train is same for all policy so we just choose one

# starts training
printn = 100    # how many epochs we print
n_epoch = int(1e2)
# look-ahead horizon
horizon = [1] # we start from 1 as 0 is contained from single policy already
t_int = time.time()
train_step = 0
valid_step = 0
for k in horizon:
    print('Horizon {0:} {1:}'.format(k, '='*10))
    for epoch in range(n_epoch):
        print('Epoch:', epoch)
        epoch_loss =0.
        # number of train batches
        n_train_batch = len(train)//batch_size
        t1 = time.time()
        for batch in iterate_minibatches(train, policy_targets, batch_size, shuffle=False):
            train_xi, train_yi = batch
            train_pxi = []
            # go through each policy
            for policy in policies:
                model = single_policies[policy]
                # forward_pass make predictions for current timestep
                p = model.forward_pass(train_xi, k)
                # combine and recreate input
                train_pxi.append(p[0])
            # update each other
            train_xi_updated = np.copy(train_xi)
            for policy in policies:
                for i in range(batch_size):
                    # only updates based on horzion
                    # how to deal with the the part where horzion can't reach????????????????????????????????????
                    train_xi_updated[i][list(range(1, sequence_length-1, k+1)), policy*4:policy*4+2]\
                        = train_pxi[policy][i][list(range(0, sequence_length-2, k+1)), :]
            # feed back into the model for training
            for policy in policies:
                train_pyi = [i[:, policy*2:policy*2+2] for i in train_yi]
                p, l, _ = model.backward_pass(train_xi_updated, train_pyi, 0)
            epoch_loss += l/n_train_batch
            train_step += 1
        # print out info
        if epoch%printn ==0:
#             # number of validation batches
#             n_val_batch = len(test_game)//batch_size
            t2 = time.time()
#             valid_loss = 0
#             for test_batch in iterate_minibatches(test_game, test_target, batch_size, shuffle=False):
#                 val_xi, val_yi = test_batch
#                 val_l, valid_sum = model.validate(val_xi, val_yi, k)

#                 model.valid_writer.add_summary(valid_sum, valid_step)
#                 valid_loss += val_l/n_val_batch
#                 valid_step += printn
#             print('Epoch {0:<4d} | loss: {1:<8.2f} | time took: {2:<.2f}s '
#                 '| validation loss: {3:<8.2f}'.format(epoch, epoch_loss, (t2-t1), valid_loss))
            print(epoch, epoch_loss, t2-t1)

    print('Total time took: {0:<.2f}hrs'.format((time.time()-t_int)/60/60))

### To do:

    - 1) Regularize the lstm
    - 2) Figure out why there are blanks in the testing
    - 3) may consider to collect those left out from the process of creating batches
    - 4) related to 3), seq_len = 3 may create null batches 

    - Split data to defending and offending, as the model for e.g. forward role in deffending and offending should be pretty different. Remove particular events, like free-throw etc.
    
    - We can use the shot clock as an indicator of when the offending and defending switches.
    
    - The cameras oprate at 25 frames per second, so in order to learn realistic motions, either we sample the 25 frames, or extend the horizon to 50 for example or even longer(this might be too computationally heavy and model would probably drift a lot).
    
    - At the moment if we don't have defending or offending sepearted, at least we need to break down the sequences from the 24 secs shot clock, since it usually stands for a change in game state. (note: shot clock sometimes is None)
    
    - Add tensorboard visualization. Add validation performance (maybe, it would take longer). 
      tensorboard --logdir=./train_logs
      
    - Start thinking about 1) joint training 2) Hidden structure 3) Smooth learning
    - from each sample to next sample theres not much change, subsample them 

### Questions

    * After a team scored and they go back to get ready for defense, is the going back trajectory pretty much random?
    * Do player swap roles during the play? e.g. a forward swapped to a guard, is the forward roles a lot different from gaurds these day? (i.e. can you differentaite a player plaing forward from guard from the game) If yes, then the hidden structure learning/sequencing is necessary. 